In [3]:
import pandas as pd
import numpy as np
import joblib
import pytesseract
from PIL import Image
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pickle

file_path = "kidney.csv"
df = pd.read_csv(file_path)

df["Stage"] = df["Stage"].apply(lambda x: 1 if x in ["Stage 3", "Stage 4", "Stage 5"] else 0)
df["Gender"] = df["Gender"].map({"Male": 1, "Female": 0})


X = df.drop(columns=["Stage"])
y = df["Stage"]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)




joblib.dump(rf_model, "kidney_rf_model.pkl")
joblib.dump(scaler, "kidney_scaler.pkl")


rf_model = joblib.load("kidney_rf_model.pkl")
scaler = joblib.load("kidney_scaler.pkl")

def extract_data_from_report(image_path):
    ocr_text = pytesseract.image_to_string(Image.open(image_path))
    
    def extract_value(pattern, default=None, cast_type=str):
        match = re.search(pattern, ocr_text)
        return cast_type(match.group(1)) if match else default

    extracted_data = {
        "Age": extract_value(r"Age:\s*(\d+)", default=50, cast_type=int),
        "Gender": extract_value(r"Gender:\s*(Male|Female)", default="Male"),
        "Blood Pressure": extract_value(r"Blood Pressure:\s*(\d+)", default=120, cast_type=int),
        "Glucose": extract_value(r"Glucose:\s*([\d.]+)", default=100.0, cast_type=float),
        "Creatinine": extract_value(r"Creatinine:\s*([\d.]+)", default=1.0, cast_type=float),
        "Urea": extract_value(r"Urea:\s*([\d.]+)", default=30.0, cast_type=float),
        "Albumin": extract_value(r"Albumin:\s*([\d.]+)", default=3.5, cast_type=float),
        "Hemoglobin": extract_value(r"Hemoglobin:\s*([\d.]+)", default=13.5, cast_type=float),
        "Potassium": extract_value(r"Potassium:\s*([\d.]+)", default=4.0, cast_type=float),
        "Sodium": extract_value(r"Sodium:\s*([\d.]+)", default=140.0, cast_type=float),
        "Calcium": extract_value(r"Calcium:\s*([\d.]+)", default=9.0, cast_type=float),
        "Phosphorus": extract_value(r"Phosphorus:\s*([\d.]+)", default=4.0, cast_type=float)
    }
    
    extracted_data["Gender"] = 1 if extracted_data["Gender"] == "Male" else 0

    return np.array([[extracted_data[feature] for feature in X.columns]])

def predict_from_report(image_path):
    report_data = extract_data_from_report(image_path)
    report_scaled = scaler.transform(report_data)
    rf_pred = rf_model.predict(report_scaled)
    rf_proba = rf_model.predict_proba(report_scaled)[0][1]

    risk_percentage = round(rf_proba * 100, 2)

    if risk_percentage >= 75:
        risk_level = "\U0001F525 High Risk of Kidney Disease"
    elif 50 <= risk_percentage < 75:
        risk_level = "⚠️ Moderate Risk"
    else:
        risk_level = "✅ Low Risk"

    print(f"""
    Conclusion: {risk_level}
    - 📊 Estimated Risk: {risk_percentage}%
    - Prediction: {"🩺 Chronic Kidney Disease (CKD)" if rf_pred[0] == 1 else "✅ No CKD Detected"}
    
    Recommendation: 💧 Stay hydrated, maintain a balanced diet, and consult a nephrologist if symptoms persist.
    """)


image_path = "kidney3.jpg"  
predict_from_report(image_path)



    Conclusion: ✅ Low Risk
    - 📊 Estimated Risk: 4.0%
    - Prediction: ✅ No CKD Detected
    
    Recommendation: 💧 Stay hydrated, maintain a balanced diet, and consult a nephrologist if symptoms persist.
    


C:\Users\Sushmidha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [4]:
import pickle


with open("kidney_rf_model.pkl", "wb") as f:
    pickle.dump(rf_model, f)


with open("kidney_scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

print("Model and scaler saved successfully using pickle!")


Model and scaler saved successfully using pickle!
